In [1]:
import nltk 
from nltk import *
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
import os
import pandas as pd
import glob
import requests
from bs4 import BeautifulSoup
import numpy as np

In [2]:
stopwords_files = glob.glob('E:/blackcoffer/20211030 Test Assignment/StopWords/*.txt') 
with open("allstopwords.txt", "w") as out_file:
    for fn in stopwords_files:
            with open(fn,encoding='latin-1') as in_file:
                contents = in_file.read()
                out_file.write(contents)

In [3]:
pos_file=open("E:/blackcoffer/20211030 Test Assignment/MasterDictionary/positive-words.txt")
positive_list=[]
for i in pos_file:
    positive_list.append(i.strip())

neg_file=open("E:/blackcoffer/20211030 Test Assignment/MasterDictionary/negative-words.txt")
negative_list=[]
for i in neg_file:
    negative_list.append(i.strip())

nltk.download('stopwords')
nltk.download('punkt')   





UnicodeDecodeError: 'utf-8' codec can't decode byte 0xef in position 3988: invalid continuation byte

In [ ]:
def extractor(val):
    extractor.words = word_tokenize(val)
    text = re.sub(r"[^a-zA-Z0-9]", " ", val.lower())
    words = text.split()
    special_char = '@_!#$%^&*()<>?/\|}{~:;.[]₹'
    extractor.total_number_of_words=len(words)
    out_list = [''.join(x for x in string if not x in special_char) for string in words]

    return out_list

In [ ]:
file_st=open("../model/allstopwords.txt",newline='',encoding='latin-1')
stopwords_input=file_st.read()
sw_data=extractor(stopwords_input)


In [ ]:
final_data=pd.read_csv('../model/input_final.csv')
final_data.columns
final_data.drop(['Unnamed: 0'],axis=1)

,URL_ID,URL
0,37,https://insights.blackcoffer.com/ai-in-healthc...
1,38,https://insights.blackcoffer.com/what-if-the-c...
2,39,https://insights.blackcoffer.com/what-jobs-wil...
3,40,https://insights.blackcoffer.com/will-machine-...
4,41,https://insights.blackcoffer.com/will-ai-repla...
...,...,...
106,146,https://insights.blackcoffer.com/blockchain-fo...
107,147,https://insights.blackcoffer.com/the-future-of...
108,148,https://insights.blackcoffer.com/big-data-anal...
109,149,https://insights.blackcoffer.com/business-anal...


In [ ]:
def Calculation(txt_val):
    global Average_size_of_word
    global Personal_pronouns_count
    global Positive_Score
    global Negative_Score
    global polarity
    global subjectivity
    global Average_sentence_length
    global complex_count
    global syllable_count_per_word
    global percentage_of_complex_words
    global word_count_after_removing_stop_words
    global fog_index
    
    data_file=extractor(txt_val)
    e_list = [x for x in data_file if x not in sw_data]
    total_words_after_cleaning=len(e_list)

    sent_p=0
    sent_n=0
    for word_s in e_list:
        if word_s in positive_list:
            sent_p+=1
        elif word_s in negative_list:
            sent_n-=1
    sent_n=sent_n*(-1)
    Positive_Score=sent_p
    Negative_Score=sent_n
    
    polarity=(sent_p-sent_n)/((sent_p+sent_n)+0.000001)
    subjectivity=(sent_p+sent_n)/((total_words_after_cleaning)+0.000001)

    total_sentences=txt_val.count(".")
    total_word=extractor.total_number_of_words
    
    Average_sentence_length=total_word/total_sentences
    
    def syllable_count(word):
        count = 0
        vowels = "aeiou"
        if word[0] in vowels:
            count += 1
        for index in range(1, len(word)):
            if word[index] in vowels and word[index - 1] not in vowels:
                count += 1
        if word.endswith("es") or word.endswith("ed"):
            count -= 1
        if count == 0:
            count += 1
        return count
    complex_count=0
    syllable_count_per_word=[]
    for x in extractor.words:
        k=syllable_count(x)
        syllable_count_per_word.append(k)
        if k>2:
            complex_count+=1
    syllable_count_per_word=sum(syllable_count_per_word)
   
    
    percentage_of_complex_words=complex_count/total_word
    
    fog_index=0.4*(Average_sentence_length+percentage_of_complex_words)

    stop_words_nltk=stopwords.words('english')
    word_count_after_removing_stop_words = len([x for x in data_file if x not in stop_words_nltk])

    
    value=0
   
    for x in extractor.words:
        pattern=r"(\b(I|we|my|ours|us)\b)"
        r1 = re.match(pattern, x)
        if r1:
            value+=1
    Personal_pronouns_count=value

    size_of_word=0
    for x in data_file:
        size_of_word=size_of_word+len(x)

    Average_size_of_word=size_of_word/total_word

In [ ]:
col_name = ['URL_ID', 'URL', 'POSITIVE SCORE', 'NEGATIVE SCORE', 'POLARITY SCORE',
       'SUBJECTIVITY SCORE', 'AVG SENTENCE LENGTH',
       'PERCENTAGE OF COMPLEX WORDS', 'FOG INDEX',
       'AVG NUMBER OF WORDS PER SENTENCE', 'COMPLEX WORD COUNT', 'WORD COUNT',
       'SYLLABLE PER WORD', 'PERSONAL PRONOUNS', 'AVG WORD LENGTH']

out_df = pd.DataFrame(columns = col_name)



for i in range(0,len(final_data)-1):
    fn="../model/data/"+str(final_data.URL_ID[i])+".txt"
    pag=open(fn,encoding="utf-8")
    pag_in=pag.read()


    Calculation(pag_in)
    data = {'URL_ID':final_data.URL_ID[i], 'URL':final_data.URL[i], 'POSITIVE SCORE':Positive_Score, 'NEGATIVE SCORE':Negative_Score, 'POLARITY SCORE':polarity,
       'SUBJECTIVITY SCORE':subjectivity, 'AVG SENTENCE LENGTH':Average_sentence_length,
       'PERCENTAGE OF COMPLEX WORDS':percentage_of_complex_words, 'FOG INDEX':fog_index,
       'AVG NUMBER OF WORDS PER SENTENCE':Average_sentence_length, 'COMPLEX WORD COUNT':complex_count, 'WORD COUNT':word_count_after_removing_stop_words,
       'SYLLABLE PER WORD':syllable_count_per_word, 'PERSONAL PRONOUNS':Personal_pronouns_count, 'AVG WORD LENGTH':Average_size_of_word,
       }
    out_df = out_df.append(data, ignore_index = True)


C:\Users\ASUS\AppData\Local\Temp\ipykernel_26040\264235517.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out_df = out_df.append(data, ignore_index = True)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_26040\264235517.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out_df = out_df.append(data, ignore_index = True)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_26040\264235517.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out_df = out_df.append(data, ignore_index = True)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_26040\264235517.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out_df = out_df.append(data, ignore_index = Tr

In [ ]:
out_df.to_excel('Output Data Structure.xlsx',index = False)